In [1]:
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
import numpy as np
from GraphST import GraphST
from GraphST.utils import clustering
from st_loading_utils import load_mPFC, load_mHypothalamus, load_her2_tumor, load_mMAMP, load_spacelhBC, load_DLPFC, load_BC, load_mVC
import time
from scipy.sparse import csr_matrix


# Run device, by default, the package is implemented on 'cpu'. We recommend using GPU.
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
iters = 5

/home/yunfei/anaconda3/envs/GraphST/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# mouse hoppocampus

In [2]:
import numpy as np
import anndata
import matplotlib.pyplot as plt

# plt.rcParams['figure.figsize'] = (30.0, 10.0)
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 26

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
"""spacel hBC"""
# the number of clusters
n_clusters = 14

dataset = 'sshippo.h5ad'

dir_ = '/home/yunfei/spatial_benchmarking/benchmarking_data/mouse_hyppocampus_slideseqv2'
ad = sc.read_h5ad(os.path.join(dir_, dataset))
aris = []
run_times = []
for iter in range(1):
    # define model
    time_start = time.time()
    model = GraphST.GraphST(ad, device=device)

    # train model
    ad = model.train()

    # print(ad)

    # set radius to specify the number of neighbors considered during refinement
    radius = 30

    tool = 'mclust' # mclust, leiden, and louvain

    # clustering
    if tool == 'mclust':
        clustering(ad, n_clusters, radius=radius, method=tool, refinement=True) # For DLPFC dataset, we use optional refinement step.
    elif tool in ['leiden', 'louvain']:
        clustering(ad, n_clusters, radius=radius, method=tool, start=0.1, end=2.0, increment=0.01, refinement=False)
    time_end = time.time()      

    # filter out NA nodes
    ad = ad[~pd.isnull(ad.obs['cluster'])]
    # ad.uns['runtime'] = time_end - time_start
    # run_times.append(time_end - time_start)
    # ad.X = csr_matrix(ad.X)

    # if not os.path.exists(os.path.join(save_dir_gt, dataset)):
    #     os.makedirs(os.path.join(save_dir_gt, dataset))
    # ad.write_h5ad(os.path.join(save_dir_gt, dataset, 'iter'+str(iter)+'.h5ad'))
    # calculate metric ARI
    print(ad.obs['domain'])
    # print(ad.obs['original_clusters'])
    ARI = metrics.adjusted_rand_score(ad.obs['domain'], ad.obs['cluster'])
    # ad.uns['ARI'] = ARI

    # print('Dataset:', dataset)
    print('ARI:', ARI)
    aris.append(ARI)

    sc.pl.spatial(ad,
            color=["mclust", "cluster"],
            title=["GraphST", "Ground Truth"],
            show=False, spot_size=20)
    plt.savefig(os.path.join("/home/yunfei/spatial_benchmarking/BenchmarkST/analysis1110/clustering/mousehippo", "hippocampus_GraphST.pdf"), bbox_inches='tight')

Begin to train ST data...


100%|██████████| 600/600 [19:37<00:00,  1.96s/it]


Optimization finished for ST data!


R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 5.4.10
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%
AAAAAAAGTCCCAA     6
AAAAAACATCTTTC    14
AAAAAACTTCTACT     4
AAAAAATTTACAAC     4
AAAAACACGGTGGT     2
                  ..
TTTTTTTCTGACGG    13
TTTTTTTCTGCAGT     4
TTTTTTTCTGCCAT    11
TTTTTTTTCGCTTA     2
TTTTTTTTTCGGTG    11
Name: domain, Length: 41770, dtype: object


KeyError: 'original_clusters'

In [ ]:
ARI = metrics.adjusted_rand_score(ad.obs['domain'], ad.obs['cluster'])
# ad.uns['ARI'] = ARI

# print('Dataset:', dataset)
print('ARI:', ARI)
aris.append(ARI)

sc.pl.spatial(ad,
        color=["mclust", "cluster"],
        title=["GraphST", "Ground Truth"],
        show=False, spot_size=20)
plt.savefig(os.path.join("/home/yunfei/spatial_benchmarking/BenchmarkST/analysis1110/clustering/mousehippo", "hippocampus_GraphST.pdf"), bbox_inches='tight')

# outdated spacel hbc

In [4]:
"""spacel hBC"""
# the number of clusters
# setting_combinations = [[10, 'human_bc_spatial_1142243F'],
#                         [10, 'human_bc_spatial_1160920F'], 
#                         [10, 'human_bc_spatial_CID4290'], 
#                         [10, 'human_bc_spatial_CID4465'], 
#                         [10, 'human_bc_spatial_CID4535'], 
#                         [10, 'human_bc_spatial_CID44971'], 
#                         [10, 'human_bc_spatial_Parent_Visium_Human_BreastCancer'], 
#                         [10, 'human_bc_spatial_V1_Breast_Cancer_Block_A_Section_1'],
#                         [10, 'human_bc_spatial_V1_Breast_Cancer_Block_A_Section_2'],
#                         [10, 'human_bc_spatial_V1_Human_Invasive_Ductal_Carcinoma'],
#                         [10, 'human_bc_spatial_Visium_FFPE_Human_Breast_Cancer']]
# for setting_combi in setting_combinations:
n_clusters = 3

dataset = 'human_bc_spatial_V1_Breast_Cancer_Block_A_Section_1'

dir_ = '/home/yunfei/spatial_benchmarking/benchmarking_data/visium_human_breast_cancer'
ad = load_spacelhBC(root_dir=dir_, section_id=dataset)

aris = []
run_times = []
for iter in range(iters):
    # define model
    time_start = time.time()
    model = GraphST.GraphST(ad, device=device)

    # train model
    ad = model.train()

    # print(ad)

    # set radius to specify the number of neighbors considered during refinement
    radius = 400

    tool = 'mclust' # mclust, leiden, and louvain

    # clustering
    if tool == 'mclust':
        clustering(ad, n_clusters, radius=radius, method=tool, refinement=True) # For DLPFC dataset, we use optional refinement step.
    elif tool in ['leiden', 'louvain']:
        clustering(ad, n_clusters, radius=radius, method=tool, start=0.1, end=2.0, increment=0.01, refinement=False)
    time_end = time.time()      

    # filter out NA nodes
    ad = ad[~pd.isnull(ad.obs['original_clusters'])]
    # ad.uns['runtime'] = time_end - time_start
    # run_times.append(time_end - time_start)
    # ad.X = csr_matrix(ad.X)

    # if not os.path.exists(os.path.join(save_dir_gt, dataset)):
    #     os.makedirs(os.path.join(save_dir_gt, dataset))
    # ad.write_h5ad(os.path.join(save_dir_gt, dataset, 'iter'+str(iter)+'.h5ad'))
    # calculate metric ARI
    print(ad.obs['domain'])
    print(ad.obs['original_clusters'])
    ARI = metrics.adjusted_rand_score(ad.obs['domain'], ad.obs['original_clusters'])
    # ad.uns['ARI'] = ARI

    # print('Dataset:', dataset)
    print('ARI:', ARI)
    aris.append(ARI)

Begin to train ST data...


100%|██████████| 600/600 [00:10<00:00, 56.26it/s]


Optimization finished for ST data!
fitting ...
  |======================================================================| 100%
AAACAAGTATCTCCCA-1    1
AAACACCAATAACTGC-1    2
AAACAGAGCGACTCCT-1    1
AAACAGGGTCTATATT-1    1
AAACAGTGTTCCTGGG-1    2
                     ..
TTGTTGTGTGTCAAGA-1    1
TTGTTTCACATCCAGG-1    2
TTGTTTCATTAGTCTA-1    2
TTGTTTCCATACAACT-1    3
TTGTTTGTGTAAATTC-1    1
Name: domain, Length: 3798, dtype: object
AAACAAGTATCTCCCA-1    Intermediate
AAACACCAATAACTGC-1           Tumor
AAACAGAGCGACTCCT-1    Intermediate
AAACAGGGTCTATATT-1          Immune
AAACAGTGTTCCTGGG-1           Tumor
                          ...     
TTGTTGTGTGTCAAGA-1    Intermediate
TTGTTTCACATCCAGG-1    Intermediate
TTGTTTCATTAGTCTA-1           Tumor
TTGTTTCCATACAACT-1    Intermediate
TTGTTTGTGTAAATTC-1           Tumor
Name: original_clusters, Length: 3798, dtype: object
ARI: -0.011941063017150786
Begin to train ST data...


100%|██████████| 600/600 [00:10<00:00, 55.39it/s]


Optimization finished for ST data!
fitting ...
  |======================================================================| 100%


KeyboardInterrupt: 